# Exercícios Perceptron

In [10]:
from sklearn import datasets
import numpy as np
import pandas as pd

# Exercicio1

Considere que w é inicializado aleatoriamente, e que o problema em questão é linearmente separável. Executamos o algoritmo do perceptron duas vezes para o mesmo conjunto de treinamento. É possível que:

* a: O número de épocas necessárias para convergir para a solução seja diferente entre as duas execuções?
* b: A fronteira (valor final de w) seja diferente entre as duas execuções?

### Respostas

* a: Sim, o numero de epocas pode ser diferente. Isso ocorre pelo fato das diferencas dos valores inicias de W. Em um caso, podemos comecar com um vetor de pesos inicias mais proximo do "otimo" antes.
* b: Sim, pesos inicias diferentes podem gerar fronteiras diferentes, visando que o algoritmo para a partir do primeiro momento que classifica todas as classes corretamente ou atinge o limite de epocas.

# Exercicio 3

Modifique o problema do exemplo Iris Setosa para manter todas as quatro características do problema original, mas mantenha só as duas primeiras classes, como feito em aula.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df["target"] = iris.target

scaler = MinMaxScaler()
iris_df[iris.feature_names] = scaler.fit_transform(iris_df[iris.feature_names])

vectors_class_0 = iris_df[iris_df["target"] == 0]
vectors_class_1 = iris_df[iris_df["target"] == 1]


vectors_class_0["target"] = - 1
vectors_class_1["target"] = + 1

vectors = np.concatenate((vectors_class_0.to_numpy(), vectors_class_1.to_numpy()))
np.random.seed(42)
np.random.shuffle(vectors) 

w =  np.array([1.0, 1.0, 1.0, 1.0, -1.0])
eta = 1

allOk = False
count, epochs = 0, 15
while count < epochs and not allOk:
    print(f"Epoca: {count + 1}")

    allOk = True
    for x in vectors:
        bias_input = np.array([1, x[0], x[1], x[2], x[3]])
        result = w.dot(bias_input)
        result = 1 if result >= 0 else -1

        if result != x[4]:
            w += (eta * x[4] * bias_input)
            allOk = False

    count += 1

print(f"Epocas necessarias: {count}")
print("Resultado final: ")
print(w)

# Exercicio 4

Modifique o problema Iris Setosa agora para considerar todas as quatro características e três classes. Como usar múltiplos perceptrons em um pipeline para gerar fronteiras nesse problema multiclasse?

### Ideia

Um caminho possivel seria criar uma pipeline com 2 perceptrons, o primeiro responsavel por identificar uma unica classe, e o segundo para identificar as duas classes restantes. Dessa forma, primeiro seria executado o processo no perceptron 1, e caso ele nao apontasse como o resultado sendo a classe em questao, o mesmo input seria executado no perceptron 2.

As duvidas desse processo sao basicamente como treinar o primeiro modelo para apenas uma classe. Como nao temos camadas conectadas, neste caso nao seria necessario o processo de backpropagation para o treinamento, cada perceptron seria treinado isoladamente.

In [ ]:
class perceptron():
    def __init__(self):
        self.w = [1.0, 1.0, 1.0, 1.0]
    

    # Remove the target from the vector, otherwise this will fail
    def fit(self, vectors, epochs):
        allOk, count = False, 0

        while count < epochs and not allOk:
            allOk = True
            for x in vectors:
                bias_input = np.append(1, x)
                result = w.dot(bias_input)
                result = 1 if result >= 0 else -1

                if result != x[4]:
                    self.w += (eta * x[4] * bias_input)
                    allOk = False

            count += 1


    # Remove the target from the vector, otherwise this will fail
    # Return one preds vector with values +1 and -1, need to adapt the result for the pipeline
    def predict(self, vectors):
        preds = []

        for x in vectors:
            bias_input = np.append(1, x)
            result = w.dot(bias_input)
            preds.append(1 if result >= 0 else -1)

        return preds

In [24]:
from sklearn.preprocessing import MinMaxScaler

iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df["target"] = iris.target

first_perc_df = iris_df[~iris_df['target'].isin([1, 2])]
second_perc_df = iris_df[~iris_df['target'].isin([0])]

# Train the first perceptron with the fisrt_perc_df
# Train the second perceptron with the second_perc_df
# Create an pipeline using both perceptrons.
#   If the result from the first perceptron was not 1, it means that its not class 0
#   So its need to run in the second perceptron.
#   +1 in the second perceptron means class 1 and -1 means class 2.
#   The ideia is basicaly create 2 frontiers, one dividing the class 0 from the other 2
#   ANd the second one dividing the last 2 classes